In [1]:
import numpy as np
import pandas as pd
import sys
import random

# Distance matrix

In [2]:
#an improved version of the read_subst_mtrx from project_3
#returns the matrix and a dictionary mapping the species name to indeces
def read_mtrx(filename):
    f = open(filename,'r')
    num_species = int(f.readline())

    dict_mat = {}
    mat = np.zeros((num_species,num_species))
    
    for i in range(0,num_species):
        line = f.readline()
        nums_in_line = line.split()
        dict_mat[nums_in_line[0]] = i
        for j in range(1,num_species +1):
            mat[i,j-1] = nums_in_line[j]
    f.close()
    return dict_mat, mat 

In [3]:
species_dict, dist = read_mtrx("C:/Users/lenab/Documents/AU/Algorithms_in_bioinformatics/Brouillons_projets/example_slide4.phy")
print(species_dict)
print(dist)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
[[0.   0.23 0.16 0.2  0.17]
 [0.23 0.   0.23 0.17 0.24]
 [0.16 0.23 0.   0.2  0.11]
 [0.2  0.17 0.2  0.   0.21]
 [0.17 0.24 0.11 0.21 0.  ]]


In [4]:
species = list(species_dict.keys())
dict_dist = {}
for i in range(len(species)) :
    dict_dist[species[i]] = dist[i]
dict_dist

{'A': array([0.  , 0.23, 0.16, 0.2 , 0.17]),
 'B': array([0.23, 0.  , 0.23, 0.17, 0.24]),
 'C': array([0.16, 0.23, 0.  , 0.2 , 0.11]),
 'D': array([0.2 , 0.17, 0.2 , 0.  , 0.21]),
 'E': array([0.17, 0.24, 0.11, 0.21, 0.  ])}

In [5]:
dist=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
print(dist)

      A     B     C     D     E
A  0.00  0.23  0.16  0.20  0.17
B  0.23  0.00  0.23  0.17  0.24
C  0.16  0.23  0.00  0.20  0.11
D  0.20  0.17  0.20  0.00  0.21
E  0.17  0.24  0.11  0.21  0.00


In [16]:
dist.loc['A','B']

0.23

# NJ algorithm

## Step 1

In [8]:
N = pd.DataFrame(columns=species, index = species)
N

,A,B,C,D,E
A,NaN,NaN,NaN,NaN,NaN
B,NaN,NaN,NaN,NaN,NaN
C,NaN,NaN,NaN,NaN,NaN
D,NaN,NaN,NaN,NaN,NaN
E,NaN,NaN,NaN,NaN,NaN


In [23]:
dist.loc['A','B']

0.23

In [6]:
def step1(dist):
    S = len(dist.columns)
    N = pd.DataFrame(columns=dist.columns, index = dist.index)
    ris = []
    for i in dist.index:
        ri = 0
        for j in dist.columns:
            ri +=dist.loc[i,j]
        ris.append(ri/(S-2))
    
    rjs = []
    for j in dist.columns:
        rj = 0
        for i in dist.index:
            rj +=dist.loc[i,j]
        rjs.append(rj/(S-2))
        
    minN = sys.maxsize
    mini = dist.index[0]
    minj = dist.columns[1]
    rimin = ris[0]
    rjmin = rjs[1]
    index_mini = 0
    index_minj = 1
    for i in range(len(dist.index)):
        for j in range(len(dist.columns)):
            ci = dist.index[i]
            cj = dist.columns[j]
            nij = dist.loc[ci,cj]-ris[i]-rjs[j]
            N.loc[ci,cj]=nij
            if ci != cj:
                if nij<minN : 
                    minN = nij
                    mini = ci
                    minj = cj
                    rimin = ris[i]
                    rjmin = rjs[j]
                    index_mini = i
                    index_minj = j
    return(index_mini, index_minj, mini,minj,minN,rimin,rjmin)


In [161]:
index_mini, index_minj, mini,minj, minN,rimin, rjmin = step1(dist)
print (index_mini, index_minj, mini,minj, minN,rimin, rjmin )

1 3 B D -0.38 0.29 0.26


## Step 2

In [30]:
nodes = list(dist.index).copy()
print(nodes)

['A', 'B', 'C', 'D', 'E']


In [7]:
def step2(mini,minj,dist):
    nodes = list(dist.index).copy()
    nodes.remove(mini)
    nodes.remove(minj)
    nodes.append('('+mini+','+minj+')')
    return nodes

In [163]:
nodes = step2('B','D',dist)
nodes

['A', 'C', 'E', '(B,D)']

## Step 3

In [164]:
# This function works for the first iteration, but not for the next ones, because we do not keep in memory previous lengths
def step3(mini, minj, dist, rimin, rjmin):
    nodes = list(dist.index).copy()
    nodes.remove(mini)
    nodes.remove(minj)
    #dik = (dist.loc[mini,minj] + rimin - rjmin)/2
    #djk = (dist.loc[mini,minj] + rjmin - rimin)/2
    dik = round((dist.loc[mini,minj] + rimin - rjmin)/2,4)
    djk = round((dist.loc[mini,minj] + rjmin - rimin)/2,4)
    nodes.append('(' + mini + ': '+ str(dik) + ','+ minj + ": "+ str(djk) + ')')
    return nodes

In [165]:
nodes_length = step3(mini, minj, dist, rimin, rjmin)
nodes_length

['A', 'C', 'E', '(B: 0.1,D: 0.07)']

Try to find a solution

In [48]:
nodes_length = list(dist.index).copy()
def step3(index_mini, index_minj, mini, minj, dist, rimin, rjmin, node_length):
    nodes = node_length.copy()
    nodes.remove(node_length[index_mini])
    nodes.remove(node_length[index_minj])
    #dik = (dist.loc[mini,minj] + rimin - rjmin)/2
    #djk = (dist.loc[mini,minj] + rjmin - rimin)/2
    dik = round((dist.loc[mini,minj] + rimin - rjmin)/2,4)
    djk = round((dist.loc[mini,minj] + rjmin - rimin)/2,4)
    nodes.append('(' + node_length[index_mini] + ': '+ str(dik) + ','+ node_length[index_minj]+ ": "+ str(djk) + ')')
    return nodes

In [197]:
step3(index_mini, index_minj, mini, minj, dist, rimin, rjmin, nodes_length)

['A', 'C', 'E', '(B: 0.1,D: 0.07)']

This second function works. => Take this one

## Step 4  

In [93]:
nodes[-1]

'(B,D)'

In [104]:
dist.loc["A",:]

A    0.00
B    0.23
C    0.16
D    0.20
E    0.17
Name: A, dtype: float64

In [111]:
dist.loc["A","B"]=460
dist

,A,B,C,D,E
A,0.00,460.00,0.16,0.20,0.17
B,0.23,0.00,0.23,0.17,0.24
C,0.16,0.23,0.00,0.20,0.11
D,0.20,0.17,0.20,0.00,0.21
E,0.17,0.24,0.11,0.21,0.00


In [117]:
col_j = dist["A"]
print(col_j)
col_j["B"]

A    0.00
B    0.23
C    0.16
D    0.20
E    0.17
Name: A, dtype: float64


0.23

In [115]:
row_i = dist.loc["A",:]
print(row_i)
row_i["E"]

A      0.00
B    460.00
C      0.16
D      0.20
E      0.17
Name: A, dtype: float64


0.17

In [30]:
# Reinitialize dist
dist=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
print(dist)

      A     B     C     D     E
A  0.00  0.23  0.16  0.20  0.17
B  0.23  0.00  0.23  0.17  0.24
C  0.16  0.23  0.00  0.20  0.11
D  0.20  0.17  0.20  0.00  0.21
E  0.17  0.24  0.11  0.21  0.00


In [216]:
k = nodes[-1]
row_mini = dist.loc[mini,:]
col_minj = dist[minj]
for i in dist.index : 
    if i != mini and i!=minj : 
        dist.loc[i,k] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
        dist.loc[k,i] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
dist.loc[k,k]=0
dist = dist.drop(index = [mini,minj], columns = [mini,minj])
dist

,A,C,E,"(B,D)"
A,0.00,0.16,0.17,0.13
C,0.16,0.00,0.11,0.13
E,0.17,0.11,0.00,0.14
"(B,D)",0.13,0.13,0.14,0.00


In [9]:
def step4(mini, minj, dist):
    k = nodes[-1]
    row_mini = dist.loc[mini,:]
    col_minj = dist[minj]
    for i in dist.index : 
        if i != mini and i!=minj : 
            dist.loc[i,k] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
            dist.loc[k,i] = (dist.loc[mini,i]+dist.loc[minj,i]-dist.loc[mini,minj])/2
    dist.loc[k,k]=0
    dist = dist.drop(index = [mini,minj], columns = [mini,minj])
    return(dist)

## Try a second step

In [54]:
### Reinitialization
nodes_length = list(dist.index).copy()
# Reinitialize dist
dist=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
print(dist)

      A     B     C     D     E
A  0.00  0.23  0.16  0.20  0.17
B  0.23  0.00  0.23  0.17  0.24
C  0.16  0.23  0.00  0.20  0.11
D  0.20  0.17  0.20  0.00  0.21
E  0.17  0.24  0.11  0.21  0.00


In [60]:
index_mini, index_minj,mini,minj, minN,rimin, rjmin = step1(dist)
print (index_mini, index_minj,mini,minj, minN,rimin, rjmin)

0 3 A (B,D) -0.3 0.23 0.19999999999999998


In [61]:
nodes = step2(mini,minj,dist)
nodes

['C', 'E', '(A,(B,D))']

In [62]:
nodes_length

['A', 'C', 'E', '(B: 0.1,D: 0.07)']

In [63]:
nodes_length = step3(index_mini, index_minj, mini, minj, dist, rimin, rjmin, nodes_length)
nodes_length

['C', 'E', '(A: 0.08,(B: 0.1,D: 0.07): 0.05)']

In [64]:
dist = step4(mini, minj, dist)
dist

,C,E,"(A,(B,D))"
C,0.00,0.11,0.08
E,0.11,0.00,0.09
"(A,(B,D))",0.08,0.09,0.00


## Termination step

In [67]:
i = nodes[0]
j = nodes[1]
m = nodes[2]
print(i,j,m)
#dvi = (dist.loc[i,j]+dist.loc[i,m]-dist.loc[j,m])/2
#dvj = (dist.loc[i,j]+dist.loc[j,m]-dist.loc[i,m])/2
#dvm = (dist.loc[i,m]+dist.loc[j,m]-dist.loc[i,j])/2
dvi = round((dist.loc[i,j]+dist.loc[i,m]-dist.loc[j,m])/2,3)
dvj = round((dist.loc[i,j]+dist.loc[j,m]-dist.loc[i,m])/2,3)
dvm = round((dist.loc[i,m]+dist.loc[j,m]-dist.loc[i,j])/2,3)
tree = "("+i+":"+str(dvi)+", "+j+":"+str(dvj)+", "+m+":"+str(dvm)+")"
print(tree)

C E (A,(B,D))
(C:0.05, E:0.06, (A,(B,D)):0.03)
